In [1]:
import numpy as np
import pandas as pd
import json
import requests
import branca
import folium

In [2]:
seoul_lib = pd.read_csv('./data/서울도서관.csv')
seoul_lib.head()

,도서관명,이용시간,휴관일,전화번호,홈페이지,자치구,주소,위도,경도
0,국립어린이청소년도서관,"평일 : 09:00~18:00, 주말 : 09:00~18:00","매월 둘째넷째 월요일, 일요일을 제외한 공휴일",02-3413-4800,http://www.nlcy.go.kr/,강남구,서울특별시 강남구 테헤란로7길 21,37.501040,127.029629
1,강남구립못골도서관,"평일 : 09:00~22:00, 주말 : 09:00~22:00",매주 화요일 및 법정공휴일,02-459-5522,http://library.gangnam.go.kr/mglib,강남구,서울특별시 강남구 자곡로 116,37.471830,127.096519
2,강남역삼푸른솔도서관,"평일 : 09:00~18:00, 주말 : 09:00~17:00",매주 화요일 및 법정 공휴일,02-2051-1178,https://library.gangnam.go.kr/ysplib/index.do,강남구,서울특별시 강남구 테헤란로8길 36. 4층,37.497091,127.032024
3,논현도서관,"평일 : 09:00~22:00, 주말 : 09:00~17:00",매주 화요일 및 법정공휴일,02-3443-7650,https://library.gangnam.go.kr/nhlib,강남구,서울특별시 강남구 학동로43길 17(논현동) 논현2동주민센터 6층,37.517475,127.037182
4,대치도서관,평일 : 09:00~18:00. 주말 : 09:00~17:00,매주 화요일 및 법정공휴일,02-565-6666,https://library.gangnam.go.kr/dchlib/index.do,강남구,서울특별시 강남구 삼성로 212 은마아파트 복지상가 2층 215호,37.496581,127.064125


In [7]:
seoul_lib.tail()

,도서관명,이용시간,휴관일,전화번호,홈페이지,자치구,주소,위도,경도
202,양원숲속도서관,"평일 : 09:00~18:00, 주말 : 토:09:00∼17:00, 일:10:00~...","매주 월요일, 법정공휴일 및 근로자의날(5월1일), 창립기념일(12월1일)",02-432-0710,http://www.jungnanglib.seoul.kr/ywlib/,중랑구,서울특별시 중랑구 송림길 172(망우동),37.606854,127.110704
203,중랑구립면목정보도서관,"평일 : 09:00~22:00(월요일은 09:00~17:00), 주말 : 09:00...","매월 쳇째, 셋째 월요일, 법정공휴일 및 근로자의날(5월1일), 창립기념일(12월1일)",02-432-4123,http://www.jungnanglib.seoul.kr/mmlib/,중랑구,서울특별시 중랑구 면목로 397(면목동),37.587478,127.087620
204,중랑구립정보도서관,"평일 : 09:00~22:00(월요일은 09:00~18:00), 주말 : 09:00...","매월 둘째, 넷째 월요일, 법정공휴일 및 근로자의날(5월1일), 창립기념일(12월1일)",02-490-9112~4,http://www.jungnanglib.seoul.kr,중랑구,서울특별시 중랑구 신내로15길 197(묵동),37.615267,127.086900
205,중랑상봉도서관,"평일 : 09:00~22:00, 주말 : 09:00~17:00","매주 월요일, 법정공휴일 및 근로자의날(5월1일), 창립기념일(12월1일)",02-2209-0094,http://www.jungnanglib.seoul.kr/sblib/index.php,중랑구,서울특별시 중랑구 동일로114길 10(상봉동),37.593104,127.080905
206,중랑숲어린이도서관,"평일 : 09:00~18:00, 주말 : 09:00~17:00","매주 월요일, 법정공휴일 및 근로자의날(5월1일), 창립기념일(12월1일)",02-6235-1150,http://www.jungnanglib.seoul.kr/suplib/index.php,중랑구,서울특별시 중랑구 망우로73길 46-1(망우동),37.602626,127.103958


In [3]:
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'

In [9]:
m = folium.Map(
    location=[37.528043, 126.980238],
    zoom_start=11
)
folium.GeoJson(
    #json.loads(requests.get(seoul_geo).text),
    seoul_geo,
    name='seoul_municipalities'
).add_to(m)

for idx in range(len(seoul_lib)):
    folium.Marker([seoul_lib['위도'][idx], seoul_lib['경도'][idx]],
                 tooltip = seoul_lib.iloc[idx, 0]+'<p>'+seoul_lib.iloc[idx, 6]+'<p>'+seoul_lib.iloc[idx, 1]+'<p>'+seoul_lib.iloc[idx, 2]).add_to(m)
#folium.LayerControl().add_to(m)

m

In [10]:
# My SQL에 서울도서관.csv 파일 저장하기

import pymysql

db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = 'guess304!',
    #database = 'books',
    charset='utf8'
)

cursor = db.cursor()
sql1 = """drop database if exists books"""
cursor.execute(sql1)
db.commit()

sql2 = """create database if not exists books"""
cursor.execute(sql2)
db.commit()

sql3 = """use books"""
cursor.execute(sql3)

sql4 = """
    create table seoul_library(
        도서관명    varchar(100)    not null,
        이용시간    varchar(100)    not null,
        휴관일      varchar(100)    not null,
        전화번호    varchar(100)    null,
        홈페이지    varchar(100)    null,
        자치구      varchar(100)    not null,
        주소        varchar(100)    not null,
        위도        float             not null,
        경도        float             not null,
        primary key(도서관명)
    );
"""
cursor.execute(sql4)
db.commit()

for i in range(len(seoul_lib)):
    name = seoul_lib.iloc[i, 0]
    time = seoul_lib.iloc[i, 1]
    offday = seoul_lib.iloc[i, 2]
    tel = seoul_lib.iloc[i, 3]
    homepage = seoul_lib.iloc[i, 4]
    state = seoul_lib.iloc[i, 5]
    address = seoul_lib.iloc[i, 6]
    lat = seoul_lib.iloc[i, 7]
    long = seoul_lib.iloc[i, 8]
    
    sql5 = """ insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        """ +"""'""" + name + """',""" + """'""" + time + """',""" + """'""" + offday + """',""" + """'""" + str(tel) + """',""" + """'""" + homepage + """',""" + """'""" + state + """',""" + """'""" + address + """',""" + """'""" + str(lat) + """',""" + """'""" + str(long) + """'""" + """
    );"""
    
    print(sql5)
    cursor.execute(sql5)
    

 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '국립어린이청소년도서관','평일 : 09:00~18:00, 주말 : 09:00~18:00','매월 둘째넷째 월요일, 일요일을 제외한 공휴일','02-3413-4800','http://www.nlcy.go.kr/','강남구','서울특별시 강남구 테헤란로7길 21','37.50103971','127.0296286'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '강남구립못골도서관','평일 : 09:00~22:00, 주말 : 09:00~22:00','매주 화요일 및 법정공휴일','02-459-5522','http://library.gangnam.go.kr/mglib','강남구','서울특별시 강남구 자곡로 116','37.47182975','127.096519'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '강남역삼푸른솔도서관','평일 : 09:00~18:00, 주말 : 09:00~17:00','매주 화요일 및 법정 공휴일','02-2051-1178','https://library.gangnam.go.kr/ysplib/index.do','강남구','서울특별시 강남구 테헤란로8길 36. 4층','37.4970911','127.0320237'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '논현도서관','평일 : 09:00~22:00, 주말 : 09:00~17:00','매주 화요일 및 법정공휴일','02-3443-7650','https://li

 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '달빛마루도서관','평일 : 09:00~18:00, 주말 : 09:00~18:00','매주 월요일, 법정공휴일','02-911-0993','http://dalbit.sblib.seoul.kr','성북구','서울특별시 성북구 종암로 167 104동 2층','37.60528229','127.0309877'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '석관동미리내도서관','평일 : 09:00~18:00, 주말 : 09:00~18:00','매주 월요일, 법정공휴일','02-960-5067','http://mirinae.sblib.seoul.kr','성북구','서울특별시 성북구 한천로66길 203','37.61350865','127.0654049'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '성북길빛도서관','평일 : 09:00~18:00, 주말 : 09:00~18:00','매주 월요일 / 일요일을 제외한 법정공휴일, 도서관사정에 의한 임시휴관일','02-6949-0031','https://www.sblib.seoul.kr/gblib','성북구','서울특별시 성북구 숭인로8길 52','37.60986006','127.0263257'
    );
 insert into seoul_library(도서관명, 이용시간, 휴관일, 전화번호, 홈페이지, 자치구, 주소, 위도, 경도) values(
        '성북정보도서관','평일 : 09:00~18:00, 주말 : 09:00~18:00','매월 첫째, 셋째주 월요일','02-962-1081','http://lib.

In [11]:
db.commit()
db.close()

In [12]:
# My SQL의 library 데이터베이스에서 seoul_library 테이블 불러와서 지도 시각화하기

db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    password = 'guess304!',
    database = 'books',
    charset='utf8'
)

cursor = db.cursor()

sql6 = """select * from seoul_library;"""
seoul_lib2 = pd.read_sql(sql6, db)
seoul_lib2

D:\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,도서관명,이용시간,휴관일,전화번호,홈페이지,자치구,주소,위도,경도
0,4.19혁명기념도서관,평일 : 08:00~22:00,일요일 및 법정공휴일,02-730-4190,http://library.419revolution.org,종로구,서울특별시 종로구 새문안로 17,37.5675,126.968
1,가락몰도서관,"평일 : 09:00~20:00, 주말 : 09:00~18:00",매주 월요일 및 법정공휴일,"02-3435-0950, 0956",http://www.splib.or.kr,송파구,서울특별시 송파구 양재대로 932 서울시농수산식품공사 가락몰 업무동 4층,37.4951,127.116
2,가양도서관,"평일 및 토요일 : 09:00~22:00, 일요일 : 09:00~18:00",매주 월요일 및 법정 공휴일,02-3665-8797,http://lib.gangseo.seoul.kr,강서구,서울특별시 강서구 양천로61길101(가양동 1493),37.5602,126.865
3,가온도서관,"평일 : 09:00~22:00, 주말 : 09:00~22:00",매주월요일 및 법정 공휴일(일요일제외),02.2230-2921,https://www.junggulib.or.kr/,중구,서울특별시 중구 동호로14길 18,37.5564,127.011
4,갈산도서관,"평일 : 09:00~22:00, 주말 : 09:00~18:00",매주 월요일 및 법정공휴일,02-2645-5919,http://lib.yangcheon.or.kr/,양천구,서울특별시 양천구 목동남로4길 48-6,37.5090,126.867
...,...,...,...,...,...,...,...,...,...
202,향기나무도서관,"평일 : 09:00∼18:00, 토 : 09:00∼17:00","매주일요일, 법정공휴일, 임시공휴일",02-2116-4448,http://www.nowonlib.kr/,노원구,서울특별시 노원구 노원로 331 노원아동복지관,37.6456,127.070
203,홍은도담도서관,"평일 : 09:00~18:00, 주말 : 09:00~17:00",매주 금요일,02-360-8640,http://lib.sdm.or.kr/,서대문구,서울특별시 서대문구 홍은중앙로 129,37.6017,126.949
204,화랑도서관,"평일 : 09:00~22:00, 토 : 09:00~18:00, 일 : 09:00~1...","매주 월요일, 법정공휴일",02-973-1318,http://www.gycenter.or.kr/,노원구,서울특별시 노원구 노원로1나길 10,37.6215,127.083
205,휘경어린이도서관,"평일 : 09:00~18:00, 주말 : 09:00~18:00","매주 월요일, 법정 공휴일",02-2248-1959,http://www.l4d.or.kr/,동대문구,서울특별시 동대문구 망우로18가길 38,37.5886,127.061


In [13]:
db.commit()
db.close()

In [14]:
url = 'https://raw.githubusercontent.com/suanlab/dataset/master'
seoul_geo = f'{url}/seoul_municipalities_geo_simple.json'

m = folium.Map(
    location=[37.528043, 126.980238],
    zoom_start=11
)
folium.GeoJson(
    #json.loads(requests.get(seoul_geo).text),
    seoul_geo,
    name='seoul_municipalities'
).add_to(m)

for idx in range(len(seoul_lib2)):
    folium.Marker([seoul_lib2['위도'][idx], seoul_lib2['경도'][idx]],
                    tooltip = seoul_lib2.iloc[idx, 0]+'<p>'+seoul_lib2.iloc[idx, 6]+'<p>'+seoul_lib2.iloc[idx, 1]+'<p>'+seoul_lib2.iloc[idx, 2]).add_to(m)
#folium.LayerControl().add_to(m)

m